In [5]:
import pandas as pd

BASE_PATH = '/home/baldasseroniluisesteban/buckets/b1/datasets/'

df = pd.read_csv(BASE_PATH + 'competencia_02_fe.csv')
df.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,865361379,202101,1,0,0,34,134,445.55,5462.66,385.70,...,4.0,0.0,-779.25,0.0,4074.0,779.25,2.0,0.0,35.19,CONTINUA
1,865361379,202102,1,0,0,34,135,525.97,5734.45,380.13,...,-3.0,0.0,-788.60,0.0,4102.0,861.70,2.0,0.0,35.19,CONTINUA
2,865361379,202103,1,0,0,34,136,2075.33,7553.68,1994.59,...,1.0,0.0,-872.05,0.0,4133.0,988.35,2.0,0.0,46.92,CONTINUA
3,865361379,202104,1,0,0,34,137,-1077.29,6202.98,-1159.95,...,2.0,0.0,-1000.21,0.0,4163.0,988.35,2.0,0.0,46.92,CONTINUA
4,865361379,202105,1,0,0,34,138,541.40,5178.31,475.75,...,5.0,0.0,-1000.21,0.0,4194.0,988.35,2.0,0.0,46.92,CONTINUA


In [7]:
# Cargar el archivo con el diccionario de estadísticas (medias, cuartiles, deciles, etc.)
df_estadisticas = pd.read_csv(BASE_PATH + 'DiccionarioDatosNulos.csv')

# Cargar el archivo con las reglas de imputación
df_reglas_imputacion = pd.read_csv(BASE_PATH + 'rank.csv')

In [8]:
import pandas as pd

def imputar_valores(df, df_estadisticas, rank_csv):
    # Iterar sobre las filas del archivo rank.csv
    for _, row in rank_csv.iterrows():
        nombre_columna = row['varname']  # Nombre de la columna a imputar
        caso_null = row['CASO_NULL']  # Regla de imputación ('media', 'mediana', etc.)
        
        # Verificar si la columna existe en el DataFrame original
        if nombre_columna not in df.columns:
            print(f"Advertencia: La columna '{nombre_columna}' no se encuentra en el DataFrame original.")
            continue
        
        # Verificar si la columna tiene estadísticas asociadas en df_estadisticas
        if nombre_columna not in df_estadisticas['columna'].values:
            print(f"Advertencia: No se encuentran estadísticas para la columna '{nombre_columna}'.")
            continue
        
        # Obtener las estadísticas correspondientes de la columna desde df_estadisticas
        stats = df_estadisticas[df_estadisticas['columna'] == nombre_columna].iloc[0]
        
        # Determinar el valor de imputación según el caso_null
        if caso_null == "media":
            valor_imputar = stats['media']
        elif caso_null == "mediana":
            valor_imputar = stats['mediana']
        elif caso_null == "cuartil_1":
            valor_imputar = stats['cuartil_1']
        elif caso_null == "cuartil_3":
            valor_imputar = stats['cuartil_3']
        elif caso_null == "deciles":
            # Aquí tomamos el primer decil, pero puedes ajustar según tus necesidades
            valor_imputar = stats['deciles'][0]  # Tomar el primer decil (ajustar si es necesario)
        else:
            # Si la regla no es conocida, por defecto se utiliza la media
            valor_imputar = stats['media']
        
        # Imputar los valores nulos en la columna correspondiente
        df.loc[df[nombre_columna].isna(), nombre_columna] = valor_imputar
    
    return df

In [4]:
# Aplicar la imputación
df = imputar_valores(df, df_estadisticas, df_reglas_imputacion)


KeyError: 'columna'

In [ ]:
# Guardar el dataset imputado
df.to_csv('dataset_imputado.csv.gz', index=False, compression='gzip')
